In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

### PD Set Options

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [18]:
dsf1 = tabula.read_pdf('Topcondoimage 10-03-2022.pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 10-17-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 10-24-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 10-10-2022.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 10-31-2022.pdf', pages='all',lattice=True)
dsf6 = tabula.read_pdf('Topcondoimage 11-07-2022.pdf', pages='all',lattice=True)

In [19]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]
df6 = dsf6[0]

In [20]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]
df6.columns = df6.iloc[0]

In [21]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])
df6 = df6.drop(df6.index[0])

In [22]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]
df6 = df6.iloc[: , 1:]

In [23]:
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [24]:
# df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [27]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [28]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,One Thousand Museum\r1000 Biscayne Blvd PH-57\rMiami,5/27/2022,9/29/2022,116,"$19,500,000.00",9200,"$2,119.57",Julian Johnston,The Corcoran Group,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd PH-57 Miami,One Thousand Museum,1000 Biscayne Blvd PH-57 Miami
1,SLS Brickell\r1300 S Miami Ave UPH3\rMiami,6/28/2022,9/29/2022,63,"$4,600,000.00",2268,"$2,028.22",MariaClaudia Fernandez,One Sotheby's Int'l Realty,David Freed,Keller Williams Miami Beach Realty,SLS Brickell 1300 S Miami Ave UPH3 Miami,SLS Brickell,1300 S Miami Ave UPH3 Miami
2,Jade Signature\r16901 Collins Ave 3705\rSunny Isles Beach,12/8/2021,9/28/2022,226,"$4,500,000.00",3405,"$1,321.59",Marina Osmacova,MIAMI VIP REALTY LLC,Steven Seigel,"Compass Florida, LLC",Jade Signature 16901 Collins Ave 3705 Sunny Isles Beach,Jade Signature,16901 Collins Ave 3705 Sunny Isles Beach
3,Green Diamond\r4775 Collins Ave 4003\rMiami Beach,4/26/2022,9/30/2022,58,"$3,550,000.00",1980,"$1,792.93",Claudia Whitechurch,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 4003 Miami Beach,Green Diamond,4775 Collins Ave 4003 Miami Beach
4,Prive\r5000 Island Estates Dr 1007\rAventura,6/30/2022,9/30/2022,29,"$3,500,000.00",2830,"$1,236.75",Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Natalia Salgado,Fortune Int'l Realty Key Biscayne Inc,Prive 5000 Island Estates Dr 1007 Aventura,Prive,5000 Island Estates Dr 1007 Aventura
5,The Setai\r101 20th St 2201\rMiami Beach,6/26/2022,9/27/2022,31,"$3,300,000.00",1198,"$2,754.59",Cecilia Solorzano,Suma Luxury Realty,Matt Lill,The Corcoran Group,The Setai 101 20th St 2201 Miami Beach,The Setai,101 20th St 2201 Miami Beach
6,Fisher Island\r19123 Fisher Island Dr 19123\rMiami Beach,4/19/2022,9/27/2022,118,"$2,430,000.00",1875,"$1,296.00",Nataly Stuart,"Socialite Realty, LLC",Gregory Gringruz,The Island,Fisher Island 19123 Fisher Island Dr 19123 Miami Beach,Fisher Island,19123 Fisher Island Dr 19123 Miami Beach
7,Murano Grande\r400 Alton Rd TH-4A\rMiami Beach,4/12/2022,9/30/2022,127,"$1,920,000.00",1912,"$1,004.18",Barbara Smith,"Golden Realty Group, Inc.",Oliver Davis,Keller Williams Miami Beach Realty,Murano Grande 400 Alton Rd TH-4A Miami Beach,Murano Grande,400 Alton Rd TH-4A Miami Beach
8,St Tropez on the Bay\r150 Sunny Isle Blvd 1-UPH3\rSunny Isles Beach,4/5/2022,9/27/2022,76,"$1,700,000.00",3020,$562.91,Bento Queiroz,"Compass Florida, LLC",Gina Nash,Aston Rose Sports And Entertai,St Tropez on the Bay 150 Sunny Isle Blvd 1-UPH3 Sunny Isles Beach,St Tropez on the Bay,150 Sunny Isle Blvd 1-UPH3 Sunny Isles Beach
9,R0each Condo\r68 SE 6th St 2603\rMiami,11/11/2021,9/27/2022,54,"$1,700,000.00",1891,$899.00,Claudia Lopez,Douglas Elliman,Ross Milroy,The Corcoran Group,R0each Condo 68 SE 6th St 2603 Miami,R,0each Condo 68 SE 6th St 2603 Miami


In [29]:
x = df['Close Date'].str.startswith('10')
df = df[x]

In [30]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
10,Residences by Armani Casa\r18975 Collins Ave 4702\rSunny Isles Beach,5/4/2022,10/11/2022,120,"$4,850,000.00",4062,"$1,527.08",Fabio Lopes,Coldwell Banker Realty,Sergio Waissmann,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 4702 Sunny Isles Beach,Residences by Armani Casa,18975 Collins Ave 4702 Sunny Isles Beach
11,Biscayne Beach\r2900 NE 7th Ave 4701\rMiami,5/17/2022,10/12/2022,125,"$3,500,000.00",2691,"$1,300.63",Monica Defortuna,Fortune International Realty,Lourdes Gutierrez,Compass Florida LLC,Biscayne Beach 2900 NE 7th Ave 4701 Miami,Biscayne Beach,2900 NE 7th Ave 4701 Miami
12,Grand Bay Tower\r430 Grand Bay Dr 1107\rKey Biscayne,7/6/2022,10/12/2022,47,"$3,450,000.00",2840,"$1,214.79",Monica Slodarz,Compass Florida LLC,Monica Slodarz,Compass Florida LLC,Grand Bay Tower 430 Grand Bay Dr 1107 Key Biscayne,Grand Bay Tower,430 Grand Bay Dr 1107 Key Biscayne
13,Ocean Three Condo\r18911 Collins Ave 601\rSunny Isles Beach,12/13/2021,10/12/2022,241,"$3,000,000.00",3945,$760.46,Vanessa Frank PA,One Sotheby's Int'l Realty,Alain Arwas,United Realty Group Inc,Ocean Three Condo 18911 Collins Ave 601 Sunny Isles Beach,Ocean Three Condo,18911 Collins Ave 601 Sunny Isles Beach
14,Commodore Club South\r199 Ocean Lane Dr 200-202\rKey Biscayne,8/1/2022,10/12/2022,43,"$2,500,000.00",2825,$884.96,Victoria Zamorano,Realty One Group Evolution,Marilyn Borroto,BHHS EWM Realty,Commodore Club South 199 Ocean Lane Dr 200-202 Key Biscayne,Commodore Club South,199 Ocean Lane Dr 200-202 Key Biscayne
15,Lake Villa 2\r733 Crandon Blvd 206\rKey Biscayne,3/5/2022,10/11/2022,191,"$2,351,000.00",2470,$951.82,Lucrecia Arana Lindemann,Dezer Platinum Realty LLC,Brigitte Nachtigall,Great Properties Int'l LLC,Lake Villa 2 733 Crandon Blvd 206 Key Biscayne,Lake Villa,2 733 Crandon Blvd 206 Key Biscayne
16,Majestic Tower\r9601 Collins Ave 1102\rBal Harbour,8/3/2022,10/12/2022,9,"$2,300,000.00",2160,"$1,064.81",Melissa Rubin,Compass Florida LLC,Joelle Oiknine,One Sotheby's Int'l Realty,Majestic Tower 9601 Collins Ave 1102 Bal Harbour,Majestic Tower,9601 Collins Ave 1102 Bal Harbour
17,Aria on the Bay\r488 NE 18th St 4312\rMiami,6/30/2022,10/13/2022,66,"$1,910,000.00",2018,$946.48,Ross Milroy,The Corcoran Group,Alessandro Saponaro,AFS Living,Aria on the Bay 488 NE 18th St 4312 Miami,Aria on the Bay,488 NE 18th St 4312 Miami
18,Porto Vita\r19925 NE 39th Pl 602\rAventura,3/28/2022,10/11/2022,123,"$1,560,000.00",2850,$547.37,Patricia Genis,One Sotheby's Int'l Realty,Patricia Genis,One Sotheby's Int'l Realty,Porto Vita 19925 NE 39th Pl 602 Aventura,Porto Vita,19925 NE 39th Pl 602 Aventura
19,O0 cean Two Condo\r19111 Collins Ave 3307\rSunny Isles Beach,4/11/2022,10/12/2022,172,"$1,525,000.00",1900,$802.63,Wendy Cohen,One Sotheby's Int'l Realty,Daniela Levtov,NG International Realty LLC,O0 cean Two Condo 19111 Collins Ave 3307 Sunny Isles Beach,O,0 cean Two Condo 19111 Collins Ave 3307 Sunny Isles Beach


In [31]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [32]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [33]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [34]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find  coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [35]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [36]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/3326340165.py:1: SettingWithCo

In [37]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [42]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,Residences by Armani Casa\r18975 Collins Ave 4702\rSunny Isles Beach,5/4/2022,10/11/2022,120,"$4,850,000.00",4062,"$1,527.08",Fabio Lopes,Coldwell Banker Realty,Sergio Waissmann,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 4702 Sunny Isles Beach,Residences by Armani Casa,18975 Collins Ave 4702 Sunny Isles Beach,"(18975 Collins Ave #4702, Sunny Isles Beach, FL 33160, USA, (25.9521284, -80.11970810000001))","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0
1,Biscayne Beach\r2900 NE 7th Ave 4701\rMiami,5/17/2022,10/12/2022,125,"$3,500,000.00",2691,"$1,300.63",Monica Defortuna,Fortune International Realty,Lourdes Gutierrez,Compass Florida LLC,Biscayne Beach 2900 NE 7th Ave 4701 Miami,Biscayne Beach,2900 NE 7th Ave 4701 Miami,"(2900 NE 7th Ave Ph 4701, Miami, FL 33137, USA, (25.8048485, -80.18598399999999))","(25.8048485, -80.18598399999999, 0.0)",25.804848,-80.185984,0.0
2,Grand Bay Tower\r430 Grand Bay Dr 1107\rKey Biscayne,7/6/2022,10/12/2022,47,"$3,450,000.00",2840,"$1,214.79",Monica Slodarz,Compass Florida LLC,Monica Slodarz,Compass Florida LLC,Grand Bay Tower 430 Grand Bay Dr 1107 Key Biscayne,Grand Bay Tower,430 Grand Bay Dr 1107 Key Biscayne,"(430 Grand Bay Dr APT 1107, Key Biscayne, FL 33149, USA, (25.6930959, -80.1579994))","(25.6930959, -80.1579994, 0.0)",25.693096,-80.157999,0.0
3,Ocean Three Condo\r18911 Collins Ave 601\rSunny Isles Beach,12/13/2021,10/12/2022,241,"$3,000,000.00",3945,$760.46,Vanessa Frank PA,One Sotheby's Int'l Realty,Alain Arwas,United Realty Group Inc,Ocean Three Condo 18911 Collins Ave 601 Sunny Isles Beach,Ocean Three Condo,18911 Collins Ave 601 Sunny Isles Beach,"(18911 Collins Ave APT 601, Sunny Isles Beach, FL 33160, USA, (25.9509344, -80.11988579999999))","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
4,Commodore Club South\r199 Ocean Lane Dr 200-202\rKey Biscayne,8/1/2022,10/12/2022,43,"$2,500,000.00",2825,$884.96,Victoria Zamorano,Realty One Group Evolution,Marilyn Borroto,BHHS EWM Realty,Commodore Club South 199 Ocean Lane Dr 200-202 Key Biscayne,Commodore Club South,199 Ocean Lane Dr 200-202 Key Biscayne,"(199 Ocean Ln Dr 200 202, Key Biscayne, FL 33149, USA, (25.6992732, -80.15724949999999))","(25.6992732, -80.15724949999999, 0.0)",25.699273,-80.157249,0.0
5,Lake Villa 2\r733 Crandon Blvd 206\rKey Biscayne,3/5/2022,10/11/2022,191,"$2,351,000.00",2470,$951.82,Lucrecia Arana Lindemann,Dezer Platinum Realty LLC,Brigitte Nachtigall,Great Properties Int'l LLC,Lake Villa 2 733 Crandon Blvd 206 Key Biscayne,Lake Villa,2 733 Crandon Blvd 206 Key Biscayne,"(733 Crandon Blvd APT 206, Key Biscayne, FL 33149, USA, (25.6872981, -80.16242369999999))","(25.6872981, -80.16242369999999, 0.0)",25.687298,-80.162424,0.0
6,Majestic Tower\r9601 Collins Ave 1102\rBal Harbour,8/3/2022,10/12/2022,9,"$2,300,000.00",2160,"$1,064.81",Melissa Rubin,Compass Florida LLC,Joelle Oiknine,One Sotheby's Int'l Realty,Majestic Tower 9601 Collins Ave 1102 Bal Harbour,Majestic Tower,9601 Collins Ave 1102 Bal Harbour,"(9601 Collins Ave APT 1102, Bal Harbour, FL 33154, USA, (25.8876563, -80.1222319))","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
7,Aria on the Bay\r488 NE 18th St 4312\rMiami,6/30/2022,10/13/2022,66,"$1,910,000.00",2018,$946.48,Ross Milroy,The Corcoran Group,Alessandro Saponaro,AFS Living,Aria on the Bay 488 NE 18th St 4312 Miami,Aria on the Bay,488 NE 18th St 4312 Miami,"(488 NE 18th St #4312, Miami, FL 33132, USA, (25.793483, -80.1869722))","(25.793483, -80.1869722, 0.0)",25.793483,-80.186972,0.0
8,Porto Vita\r19925 NE 39th Pl 602\rAventura,3/28/2022,10/11/2022,123,"$1,560,000.00",2850,$547.37,Patricia Genis,One Sotheby's Int'l Realty,Patricia Genis,One Sotheby's Int'l Realty,Porto Vita 19925 NE 39th Pl 602 Aventura,Porto Vita,19925 NE 3

In [41]:
df.at[9,'building_name']=('Ocean Two')
df.at[15,'building_name']=('101 Sunrise')
df.at[19,'building_name']=('Brickell Bay')
df.at[29,'building_name']=('Three Tequesta')
df.at[39,'building_name']=('Brickell Flatiron')
df.at[41,'building_name']=('1 Hotel & Homes')
df.at[44,'building_name']=('Marina Palms Residences')

# df.at[9,'address_only']=('Vizcayne South Condo 253 NE 2nd 4408 Miami')
# df.at[19,'address_only']=('Trump Royale 18201 Collins Ave 806 Sunny Isles Beach')
# df.at[29,'address_only']=('Trump Royale 18201 Collins Ave 3809 Sunny Isles Beach')
# df.at[39,'address_only']=('South Pointe Towers 400 S Pointe Dr 906 Miami Beach')
# df.at[45,'address_only']=('The Fairchild 3581 E Glencoe St 408 Miami')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [43]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'October 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [44]:
m.save('index.html')

In [ ]:
### Clean DF for graphic ###

In [45]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [46]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/310194361.py:4: SettingWithCopyWarning: 
A value is t

In [47]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [48]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/1544171665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_4936/1544171665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])


In [50]:
# df.sort_values('Sale Price int',ascending=True)

In [ ]:
# df.sort_values('days_on_market',ascending=True)

### Sale price sum

In [51]:
# "${:,.2f}".format(df['Sale Price int'].sum())

### Sale price average

In [ ]:
# "${:,.2f}".format(df['Sale Price int'].mean())

### Price per sf average

In [ ]:
# "${:,.2f}".format(df['price_per_sqft_int'].mean())

### Price per square foot min vs. max

In [ ]:
# "${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

In [ ]:
# "${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

### Sale price min and max

In [ ]:
# "${:,.2f}".format(df['Sale Price int'].max(axis=0))

In [ ]:
# "${:,.2f}".format(df['Sale Price int'].min(axis=0))

In [ ]:
# len(df)

In [ ]:
# df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
# df = df.replace('\n',' ',regex=True)
# df

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [52]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [53]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesOctober2022
